## Crawling

In [3]:
import time

In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [6]:
# abrir Google Chrome
driver = webdriver.Chrome(r"C:\Users\manuel.lopez\Documents\chromedriver.exe")

In [7]:
# how to treat the text of each column
treatment_battle= {
    1: lambda win_local: 1 if win_local == 'won' else 0,
    3: lambda name: name.replace('\n', ' '),
    5: lambda points: points.replace(' ', ''),
    7: lambda points: points.replace(' ', ''),
    8: lambda x: x,
    9: lambda x: x,
    10: lambda comp: comp.replace('\n', ' '),
    11: lambda x: x,
    12: lambda x: x
}

columns_battle = ['wins', 'opponent', 'local_points', 'opponent_points', 'duration', 'date', 'event', 'category', 'round', 'has_video']

In [28]:
def get_all_battles(driver, profile_id):
    # todo: extraer mas de una página
    profile_web = 'https://judobase.ijf.org/#/competitor/profile/' + profile_id
    numero_petes = 0
    driver.get(profile_web)
    try:
        driver.maximize_window()
    except:
        pass
    time.sleep(1)
    
    profile_name = driver.find_element_by_class_name('title').text.replace('\n', ' ')
    print(f'Extracting info from: {profile_name}')
    
    # ir a Contests
    driver.find_element_by_xpath('//a[@data-view="contests"]').click()
    time.sleep(1)

    tabla = driver.find_element_by_tag_name('tbody')
    
    assert tabla.get_attribute('role') == 'alert', 'Algo ha cambiado en la web, cuidado'
    
    battles = []
    
    filas = tabla.find_elements_by_tag_name('tr')
    
    for row in filas:
        battle = []
        for index, value in enumerate(row.find_elements_by_tag_name('td')):
            if index in treatment_battle.keys():
                # print(index, value.text)
                battle.append(treatment_battle[index](value.text))
            elif index == 13:
                # tiene o no video
                has_video = len(value.find_elements_by_tag_name('div')) > 0
        
        battle.append(has_video)
        
        battles.append(battle)
    
    battles = pd.DataFrame(battles, columns=columns_battle)
    
    # extraer urls
    print('Extracting video URLs')
    contests_url = driver.current_url
    play_buttons = driver.find_elements_by_xpath("//div[contains(@class, 'btn') and contains(@class, 'btn-sm') and contains(@class, 'btn-default')]")
    urls = list()
    for play_index in range(len(play_buttons)):
        while True:
            try:
                # hay que hacerlo asi porque el loop pierde el norte al clickear
                driver.get(contests_url)
                time.sleep(.1)
                play = driver.find_elements_by_xpath("//div[contains(@class, 'btn') and contains(@class, 'btn-sm') and contains(@class, 'btn-default')]")[play_index]
                play.click()
                time.sleep(.1)
                urls.append(driver.current_url)
                break
            except:
                numero_petes += 1
                pass
            
    battles.loc[battles.has_video, 'url_video'] = urls
    
    battles['local'] = profile_name
    
    # reorder columns
    cols = battles.columns.tolist()
    new_cols = [cols[0], cols[-1]] + cols[1:-1]
    battles = battles[new_cols]
    
    return battles, numero_petes

In [29]:
battles, petes = get_all_battles(driver, '3238')

Extracting info from: LOPEZ SHERIFF Estrella
Extracting video URLs


In [30]:
battles.head()

,wins,local,opponent,local_points,opponent_points,duration,date,event,category,round,has_video,url_video
0,0,LOPEZ SHERIFF Estrella,PUPP Reka,0,10,05:00,16 Nov 2018,Grand Prix The Hague 2018,-52 kg,Round 2,True,https://judobase.ijf.org/#/competition/contest...
1,1,LOPEZ SHERIFF Estrella,JIMENEZ Kristine,10,0,05:00,16 Nov 2018,Grand Prix The Hague 2018,-52 kg,Round 1,True,https://judobase.ijf.org/#/competition/contest...
2,1,LOPEZ SHERIFF Estrella,GNETO Astride,100,0,00:27,23 Feb 2018,Grand Slam Dusseldorf 2018,-52 kg,Bronze,True,https://judobase.ijf.org/#/competition/contest...
3,1,LOPEZ SHERIFF Estrella,TSCHOPP Evelyne,100,0,00:38,23 Feb 2018,Grand Slam Dusseldorf 2018,-52 kg,Repechage,True,https://judobase.ijf.org/#/competition/contest...
4,0,LOPEZ SHERIFF Estrella,MIRANDA Erika,0,10,00:44,23 Feb 2018,Grand Slam Dusseldorf 2018,-52 kg,Quarter-Final,True,https://judobase.ijf.org/#/competition/contest...


In [31]:
petes

1

In [96]:
df.head()

,wins,local,opponent,local_points,opponent_points,duration,date,event,round,has_video,url_video
0,1,BUCHARD Amandine,Charline VAN SNICK,10,0,00:00,20 Sep 2018,World Championships Baku 2018,Bronze,True,https://judobase.ijf.org/#/competition/contest...
1,0,BUCHARD Amandine,Uta ABE,0,100,03:34,20 Sep 2018,World Championships Baku 2018,Semi-Final,True,https://judobase.ijf.org/#/competition/contest...
2,1,BUCHARD Amandine,Gefen PRIMO,10,0,00:00,20 Sep 2018,World Championships Baku 2018,Quarter-Final,True,https://judobase.ijf.org/#/competition/contest...
3,1,BUCHARD Amandine,Joana RAMOS,110,0,00:14,20 Sep 2018,World Championships Baku 2018,Round 3,True,https://judobase.ijf.org/#/competition/contest...
4,1,BUCHARD Amandine,Shugen WU,110,0,02:04,20 Sep 2018,World Championships Baku 2018,Round 2,True,https://judobase.ijf.org/#/competition/contest...


## Extrae varios perfiles

In [9]:
web_66kg = 'https://judobase.ijf.org/#/wrl/2/simple'

In [10]:
driver.get(web_66kg)

try:
    driver.maximize_window()
except:
    pass
time.sleep(1)

In [11]:
tabla = driver.find_element_by_tag_name('tbody')
assert tabla.get_attribute('role') == 'alert', 'Algo ha cambiado en la web, cuidado'

In [12]:
# how to treat the text of each column
treatment_women = {
    0: lambda position: int(position),
    2: lambda name: ' '.join(name.split('\n')[::-1]),
    3: lambda x: x,
    4: lambda wrl_points: int(wrl_points)
}

In [13]:
women = []
for row in tabla.find_elements_by_tag_name('tr'):
    woman = []
    for index, value in enumerate(row.find_elements_by_tag_name('td')):
        if index in treatment_women.keys():
            woman.append(treatment_women[index](value.text))
        elif index == 1:
            profile_web = value.find_element_by_tag_name('img').get_attribute('src')
            cut_from, cut_to = profile_web[::-1].find('/'), profile_web[::-1].find('.')
            profile_id = profile_web[-cut_from : -cut_to - 1]
    
    woman.append(profile_id)
    
    women.append(woman)

In [14]:
women = pd.DataFrame(women, columns=['wrl_position', 'name', 'country', 'wrl_points', 'profile_id'])

In [15]:
women.head()

,wrl_position,name,country,wrl_points,profile_id
0,1,ABE Hifumi,JPN,6100,13208
1,2,MARGVELASHVILI Vazha,GEO,5306,9546
2,3,FLICKER Tal,ISR,4268,3872
3,4,AN Baul,KOR,4250,9713
4,5,SHMAILOV Baruch,ISR,4063,17286


In [33]:
battles_all = list()

In [34]:
petes_all = list()

In [35]:
for index, girl_id in enumerate(women.profile_id):
    print(index)
    battles_girl, petes = get_all_battles(driver, girl_id)

    battles_all.append(battles_girl)
    petes_all.append(petes)

0
Extracting info from: ABE Hifumi
Extracting video URLs
1
Extracting info from: MARGVELASHVILI Vazha
Extracting video URLs
2
Extracting info from: FLICKER Tal
Extracting video URLs
3
Extracting info from: AN Baul
Extracting video URLs
4
Extracting info from: SHMAILOV Baruch
Extracting video URLs
5
Extracting info from: ZANTARAIA Georgii
Extracting video URLs
6
Extracting info from: GANBOLD Kherlen
Extracting video URLs
7
Extracting info from: GOMBOC Adrian
Extracting video URLs
8
Extracting info from: ZHUMAKANOV Yeldos
Extracting video URLs
9
Extracting info from: CARGNIN Daniel
Extracting video URLs
10
Extracting info from: SERIKZHANOV Yerlan
Extracting video URLs
11
Extracting info from: PULIAEV Mikhail
Extracting video URLs
12
Extracting info from: SHAMILOV Yakub
Extracting video URLs
13
Extracting info from: MARUYAMA Joshiro
Extracting video URLs
14
Extracting info from: VIERU Denis
Extracting video URLs
15
Extracting info from: CHIBANA Charles
Extracting video URLs
16
Extracting 

In [36]:
len(battles_all)

100

In [38]:
petes_all[:10]

[2, 5, 7, 6, 6, 8, 6, 4, 6, 2]

In [39]:
battles_all[0].head()

,wins,local,opponent,local_points,opponent_points,duration,date,event,category,round,has_video,url_video
0,1,ABE Hifumi,SERIKZHANOV Yerlan,100,0,02:47,20 Sep 2018,World Championships Baku 2018,-66 kg,Final,True,https://judobase.ijf.org/#/competition/contest...
1,1,ABE Hifumi,AN Baul,10,0,02:38,20 Sep 2018,World Championships Baku 2018,-66 kg,Semi-Final,True,https://judobase.ijf.org/#/competition/contest...
2,1,ABE Hifumi,ZANTARAIA Georgii,100,0,03:10,20 Sep 2018,World Championships Baku 2018,-66 kg,Quarter-Final,True,https://judobase.ijf.org/#/competition/contest...
3,1,ABE Hifumi,VIERU Denis,10,0,00:00,20 Sep 2018,World Championships Baku 2018,-66 kg,Round 4,True,https://judobase.ijf.org/#/competition/contest...
4,1,ABE Hifumi,MEDVES Matteo,100,0,02:57,20 Sep 2018,World Championships Baku 2018,-66 kg,Round 3,True,https://judobase.ijf.org/#/competition/contest...


In [40]:
df = pd.concat(battles_all)

In [43]:
df.shape

(4236, 12)

In [44]:
df.to_csv('data66kg.csv')

### Más atributos

In [194]:
[at for at in dir(value) if 'find' not in at and at[0]!='_']

['clear',
 'click',
 'get_attribute',
 'get_property',
 'id',
 'is_displayed',
 'is_enabled',
 'is_selected',
 'location',
 'location_once_scrolled_into_view',
 'parent',
 'rect',
 'screenshot',
 'screenshot_as_base64',
 'screenshot_as_png',
 'send_keys',
 'size',
 'submit',
 'tag_name',
 'text',
 'value_of_css_property']

## Más funcionalidades

In [117]:
driver.back()

In [ ]:
nameRestaurant.clear()
nameRestaurant.send_keys(nameCity)
nameRestaurant.send_keys(Keys.RETURN)

html_txt = driver.page_source


driver.find_elements_by_xpath("/html/body/div[10]/div[8]/div")
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

soup = BeautifulSoup(html_txt, "lxml") 

sec.find("div",class_="popIndex rebrand popIndexDefault").get_text(strip=True) # extraccion